In [20]:
import pandas as pd
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import matplotlib.pyplot as plt
from time import sleep
import inspect
import os
from bs4 import BeautifulSoup
import requests


# Get the search term and tracking period from the user
search_term = input("Please enter the name of the product you want to search: ")
months =input("Please enter the number of months you want to track the product: ")

# To ensure that the user enters a non-string value 
while not months.isdigit():
    print("Warning: Please enter a valid integer value for the number of months.")
    months = input("Please enter the number of months you want to track the product: ")
months = int(months)


# Start the web driver and go to the Hepsiburada homepage
options = uc.ChromeOptions()
options.add_argument('--blink-settings=imagesEnabled=false') # disable images for loading of page faster
options.add_argument('--disable-notifications')
prefs = {"profile.default_content_setting_values.notifications" : 2}
options.add_experimental_option("prefs",prefs)
driver = uc.Chrome(options=options)

url = 'https://www.hepsiburada.com/'
driver.get(url)
wait = WebDriverWait(driver, 15)

# close cookies bar
wait.until(EC.element_to_be_clickable((By.ID, 'onetrust-accept-btn-handler'))).click()

# Enter the search term in the search box and press Enter
search_box = wait.until(EC.element_to_be_clickable((By.CLASS_NAME, 'theme-IYtZzqYPto8PhOx3ku3c')))
search_box.send_keys(search_term + Keys.RETURN)



# load all products
number_of_products = int(wait.until(EC.visibility_of_all_elements_located((By.CLASS_NAME, 'searchResultSummaryBar-AVnHBWRNB0_veFy34hco')))[1].text)
### visibility_of_all_elements_located is a wait strategy in Selenium that checks if all elements of a certain type are visible on the page and waits until they become visible before continuing.


number_of_loaded_products = 0
while number_of_loaded_products < number_of_products:
    loaded_products = wait.until(EC.visibility_of_all_elements_located((By.CSS_SELECTOR, 'li[class*=productListContent][id]')))
    number_of_loaded_products = len(loaded_products)
    driver.execute_script('arguments[0].scrollIntoView({block: "center", behavior: "smooth"});', loaded_products[-1])

# Get the link, name, price and seller of all the products
product = {key:[] for key in ['name','price','seller','url']}
product['name']  = [h3.text for h3 in driver.find_elements(By.CSS_SELECTOR, 'h3[data-test-id=product-card-name]')]
product['url']   = [a.get_attribute('href') for a in driver.find_elements(By.CSS_SELECTOR, 'a[class*=ProductCard]')]
product['price'] = [float(div.text.replace('TL','').replace(',','.')) for div in driver.find_elements(By.CSS_SELECTOR, 'div[data-test-id=price-current-price]')]
for i,url in enumerate(product['url']):
    print(f'Search seller names {i+1}/{number_of_loaded_products}', end='\r')
    driver.get(url)
    product['seller'] += [wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR, '.seller a'))).text]
    product['url'][i] = driver.current_url # useful to replace some long urls
    
# Sort by price in ascending order
import pandas as pd
product_list = pd.DataFrame(product).sort_values(by='price').to_dict('list')

print(f"\nThe product selected from the search results is:"+
      f"\nname:   {product_list['name'][0]}"+
      f"\nprice:  {product_list['price'][0]}"+
      f"\nseller: {product_list['seller'][0]}"+
      f"\nurl:    {product_list['url'][0]}")
    
 
product = pd.DataFrame(product).sort_values(by='price')
while True:
    save_option = input("Would you like to save the product data as Excel file? (yes or no): ").lower()
    if save_option == 'yes':
        product.to_excel('product_data.xlsx', index=False)
        print("The product data has been saved as Excel files.")
        break
    elif save_option == 'no':
        print("The product data will not be saved.")
        break
    else:
        print("Please enter a valid input (yes or no).")



# Seçilen ürün sayfasına gidin
driver.get(product_list['url'][0])

# Fiyatları alın
prices = []
dates = []
while len(prices) < months:
    price_elems = driver.find_elements(By.XPATH, "//div[@class='price-area']//strong[@itemprop='price']")
    date_elems = driver.find_elements(By.XPATH, "//div[@class='product-info']//span[@class='product-info-date']")
    
    for price_elem, date_elem in zip(price_elems, date_elems):
        price = float(price_elem.text.replace('.', '').replace(',', '.'))
        date = pd.to_datetime(date_elem.text, format='%d %B %Y')
        prices.append(price)
        dates.append(date)
        
    next_button = driver.find_element(By.XPATH, "//a[@class='page-next']")
    if 'disabled' in next_button.get_attribute('class'):
        break
    else:
        driver.execute_script("arguments[0].click();", next_button)
        

# DataFrame oluşturun ve son üç ayın verilerini seçin
df = pd.DataFrame({'Tarih': dates, 'Fiyat': prices})
start_date = pd.Timestamp.today() - pd.DateOffset(months=months)
end_date = pd.Timestamp.today()
df = df.loc[(df['Tarih'] >= start_date) & (df['Tarih'] <= end_date)]

# Grafik oluşturun
plt.plot(df['Tarih'], df['Fiyat')
plt.title('Son {} Ayın {} Fiyat Takibi'.format(months, product_name))


Please enter the name of the product you want to search:  pınar süt 1lt
Please enter the number of months you want to track the product:  3


Search seller names 35/35
The product selected from the search results is:
name:   Pınar Denge Laktozsuz Süt 1 L
price:  24.5
seller: PınarOnline
url:    https://www.hepsiburada.com/pinar-denge-laktozsuz-sut-1-l-p-ZYPINAR153103445?magaza=P%C4%B1narOnline


Would you like to save the product data as Excel file? (yes or no):  Yes


The product data has been saved as Excel files.


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//a[@class='page-next']"}
  (Session info: chrome=110.0.5481.178)
